In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

# 模型测试功能类
class ModelTester:
    """用于加载和测试神经网络模型的工具类"""
    
    @staticmethod
    def load_cifar10_batch(file):
        """加载CIFAR-10数据集批次"""
        with open(file, 'rb') as fo:
            batch = pickle.load(fo, encoding='bytes')
        data = batch[b'data']
        labels = batch[b'labels']
        return data, labels
    
    @staticmethod
    def preprocess_data(data, mean=None, std=None):
        """数据标准化处理"""
        data = data.astype(np.float32) / 255.0

        if mean is None:
            mean = np.mean(data, axis=0)
        if std is None:
            std = np.std(data, axis=0)

        std[std == 0] = 1.0
        
        normalized_data = (data - mean) / std
        return normalized_data, mean, std
    
    @staticmethod
    def to_one_hot(labels, num_classes=10):
        """标签转为one-hot编码"""
        one_hot = np.zeros((labels.shape[0], num_classes))
        one_hot[np.arange(labels.shape[0]), labels] = 1
        return one_hot

# 激活函数类
class Activation:
    @staticmethod
    def relu(x):
        return np.maximum(0, x)
    
    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
    
    @staticmethod
    def tanh(x):
        return np.tanh(x)
    
    @staticmethod
    def softmax(x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)

# 三层神经网络模型
class ThreeLayerNN:
    def __init__(self, input_size, hidden_size, output_size, activation='relu'):
        """初始化神经网络"""
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # 设置激活函数
        if activation == 'relu':
            self.activation = Activation.relu
        elif activation == 'sigmoid':
            self.activation = Activation.sigmoid
        elif activation == 'tanh':
            self.activation = Activation.tanh
        else:
            raise ValueError(f"不支持的激活函数: {activation}")
        
        # 初始化权重和偏置
        self.W1 = np.zeros((input_size, hidden_size))
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.zeros((hidden_size, output_size))
        self.b2 = np.zeros(output_size)
    
    def forward(self, X):
        """前向传播"""
        # 第一层: 输入 -> 隐藏层
        Z1 = np.dot(X, self.W1) + self.b1
        A1 = self.activation(Z1)
        
        # 第二层: 隐藏层 -> 输出
        Z2 = np.dot(A1, self.W2) + self.b2
        A2 = Activation.softmax(Z2)
        
        return A2
    
    def predict(self, X):
        """预测类别标签"""
        probabilities = self.forward(X)
        return np.argmax(probabilities, axis=1)
    
    def load_weights(self, file_path):
        """从文件加载模型权重"""
        weights = np.load(file_path)
        self.W1 = weights['W1']
        self.b1 = weights['b1']
        self.W2 = weights['W2']
        self.b2 = weights['b2']
        print(f"已加载权重。W1形状: {self.W1.shape}, W2形状: {self.W2.shape}")

def test_model(model_path, test_data_path, output_dir='./test_results'):
    """
    测试神经网络模型性能
    
    参数:
    model_path: 模型权重文件路径
    test_data_path: 测试数据文件路径
    output_dir: 输出结果的目录
    """
    # 创建输出目录（如果不存在）
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 加载测试数据
    print("加载测试数据...")
    tester = ModelTester()
    test_data, test_labels = tester.load_cifar10_batch(test_data_path)
    
    # 数据预处理
    test_data, _, _ = tester.preprocess_data(test_data)
    print(f"测试数据形状: {test_data.shape}")
    
    # 创建模型
    input_size = test_data.shape[1]
    hidden_size = 256  # 与训练时使用的隐藏层大小相同
    output_size = 10   # CIFAR-10有10个类别
    
    model = ThreeLayerNN(input_size, hidden_size, output_size, activation='relu')
    
    # 加载模型权重
    print(f"从 {model_path} 加载模型权重...")
    model.load_weights(model_path)
    
    # 测试模型
    print("评估模型性能...")
    predictions = model.predict(test_data)
    accuracy = np.mean(predictions == test_labels)
    print(f"测试准确率: {accuracy:.4f}")
    
    # 可视化一些预测结果
    num_samples_to_show = min(10, test_data.shape[0])
    plt.figure(figsize=(15, 8))
    
    for i in range(num_samples_to_show):
        # 获取原始图像数据
        img = test_data[i].reshape(3, 32, 32).transpose(1, 2, 0)
        
        # 标准化为可视化
        img = (img - img.min()) / (img.max() - img.min())
        
        # 获取预测结果和真实标签
        pred = predictions[i]
        true_label = test_labels[i]
        
        # 显示图像
        plt.subplot(2, 5, i + 1)
        plt.imshow(img)
        plt.title(f"预测: {pred}, 真实: {true_label}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'prediction_samples.png'))
    
    # 创建混淆矩阵
    conf_matrix = np.zeros((output_size, output_size), dtype=int)
    for i in range(len(test_labels)):
        conf_matrix[test_labels[i], predictions[i]] += 1
    
    # 可视化混淆矩阵
    plt.figure(figsize=(10, 8))
    plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('混淆矩阵')
    plt.colorbar()
    
    plt.xticks(np.arange(output_size), np.arange(output_size))
    plt.yticks(np.arange(output_size), np.arange(output_size))
    plt.xlabel('预测标签')
    plt.ylabel('真实标签')
    
    # 在混淆矩阵中添加数值
    thresh = conf_matrix.max() / 2
    for i in range(output_size):
        for j in range(output_size):
            plt.text(j, i, str(conf_matrix[i, j]),
                    horizontalalignment="center",
                    color="white" if conf_matrix[i, j] > thresh else "black")
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
    
    # 计算每个类别的准确率
    class_accuracies = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
    
    # 可视化每个类别的准确率
    plt.figure(figsize=(10, 6))
    plt.bar(np.arange(output_size), class_accuracies)
    plt.xticks(np.arange(output_size), np.arange(output_size))
    plt.ylim(0, 1.0)
    plt.xlabel('类别')
    plt.ylabel('准确率')
    plt.title('每个类别的准确率')
    
    # 在柱状图上添加具体数值
    for i, acc in enumerate(class_accuracies):
        plt.text(i, acc + 0.02, f'{acc:.2f}', ha='center')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'class_accuracies.png'))
    
    print("测试完成！结果已保存到", output_dir)
    return accuracy

def visualize_model_weights(model_path, output_dir='./test_results'):
    """
    可视化模型权重
    
    参数:
    model_path: 模型权重文件路径
    output_dir: 输出结果的目录
    """
    # 创建输出目录（如果不存在）
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 创建模型并加载权重
    input_size = 3072  # 32x32x3
    hidden_size = 256
    output_size = 10
    
    model = ThreeLayerNN(input_size, hidden_size, output_size)
    model.load_weights(model_path)
    
    # 可视化第一层权重
    W1 = model.W1  # 形状: [3072, hidden_size]
    
    # 对于CIFAR-10，输入为32x32x3（展平为3072）
    input_shape = (32, 32, 3)
    n_features = W1.shape[1]  # 隐藏神经元数量
    
    # 为第一层创建图形
    plt.figure(figsize=(15, 15))
    plt.suptitle("第一层权重", fontsize=16, y=0.98)
    
    # 确定可视化的网格大小
    grid_size = int(np.ceil(np.sqrt(n_features)))
    
    # 对权重进行归一化以便更好地可视化
    W1_min, W1_max = W1.min(), W1.max()
    
    # 将每个隐藏神经元的权重绘制为图像
    for i in range(min(n_features, grid_size**2)):
        ax = plt.subplot(grid_size, grid_size, i + 1)
        
        # 将权重重塑为输入图像的维度
        weight_img = W1[:, i].reshape(input_shape)
        
        # 归一化到[0, 1]以便可视化
        weight_img = (weight_img - W1_min) / (W1_max - W1_min)
        
        # 显示权重图像
        plt.imshow(weight_img)
        plt.axis('off')
        
        # 在每个子图周围添加边框
        for spine in ax.spines.values():
            spine.set_visible(True)
            spine.set_color('black')
            spine.set_linewidth(0.5)
    
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(output_dir, 'weights_layer1.png'))
    
    # 可视化第二层权重
    W2 = model.W2  # 形状: [hidden_size, output_size]
    
    # 为第二层创建图形
    plt.figure(figsize=(15, 8))
    plt.suptitle("第二层权重", fontsize=16, y=0.98)
    
    # 对权重进行归一化以便更好地可视化
    W2_min, W2_max = W2.min(), W2.max()
    
    # 将每个输出神经元的权重绘制为图像
    for i in range(output_size):
        plt.subplot(2, 5, i + 1)
        
        # 获取此输出神经元的权重
        weight_data = W2[:, i]
        
        # 将权重重塑为矩形图像以便显示
        side_length = int(np.ceil(np.sqrt(len(weight_data))))
        
        # 填充权重以形成方形网格
        padding = side_length**2 - len(weight_data)
        if padding > 0:
            weight_data = np.hstack([weight_data, np.zeros(padding)])
        
        # 重塑为方形网格
        weight_img = weight_data.reshape(side_length, side_length)
        
        # 归一化到[0, 1]以便可视化
        weight_img = (weight_img - W2_min) / (W2_max - W2_min)
        
        # 显示权重图像
        plt.imshow(weight_img, cmap='viridis')
        plt.axis('off')
        plt.title(f"类别 {i}", fontsize=10)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'weights_layer2.png'))
    
    print("权重可视化完成！结果已保存到", output_dir)

if __name__ == "__main__":
    # 设置参数
    model_weights_path = "./best_model.npz"  # 权重文件路径
    test_data_path = "./cifar-10-batches-py/test_batch"  # 测试数据路径
    output_directory = "./model_test_results"  # 结果输出目录
    
    # 测试模型
    test_accuracy = test_model(model_weights_path, test_data_path, output_directory)
    
    # 可视化模型权重
    visualize_model_weights(model_weights_path, output_directory)
    
    print(f"测试已完成。最终测试准确率: {test_accuracy:.4f}")

加载测试数据...


FileNotFoundError: [Errno 2] No such file or directory: './cifar-10-batches-py/test_batch'